### Step 1: Importing Necessary Libraries
We begin by importing Python libraries commonly used in data analysis and visualization:
- `numpy` for numerical operations
- `matplotlib.pyplot` for plotting graphs
- `pandas` for handling CSV data, which is especially useful for tabular data such as redshift catalogs

> Tip: If you haven't used `pandas` before, it's worth learning as it offers powerful tools to manipulate and analyze structured datasets.


For reading big csv files, one can use numpy as well as something called "pandas". We suggest to read pandas for CSV file reading and use that

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.constants import G, c
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u


Before we begin calculations, we define key physical constants used throughout:

- $ H_0 $: Hubble constant, describes the expansion rate of the Universe.
- $c$ : Speed of light.
-  $G$: Gravitational constant.
- $q_0$ : Deceleration parameter, used for approximate co-moving distance calculations.

We will use **`astropy.constants`** to ensure unit consistency and precision.

In [ ]:
# Constants:
H_0 = cosmo.H0.value  # Hubble constant in km/s/Mpc
c_light = c.to('km/s').value  # Speed of light in km/s
G_const = G.to('pc3/(kg*s2)').value  # Gravitational constant in pc^3 kg^-1 s^-2
q0 = -0.534  # Deceleration parameter (assumed from Planck fit KEEP it as it is)

print(f"H_0 = {H_0} km/s/Mpc")
print(f"c = {c_light} km/s")
print(f"G = {G_const} pc^3 kg^-1 s^-2")
print(f"q_0 = {q0}")

Read the csv data into the python using the method below

In [ ]:
# Update this path to point to your actual CSV file
csv_file_path = "your_data_file.csv"  # Replace with your actual file path
df = pd.read_csv(csv_file_path)
print(f"Data loaded: {len(df)} rows")
print("Columns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

### 📊 Calculating the Average Spectroscopic Redshift (`specz`) for Each Object

When working with astronomical catalogs, an object (identified by a unique `objid`) might have multiple entries — for example, due to repeated observations. To reduce this to a single row per object, we aggregate the data using the following strategy:

```python
averaged_df = df.groupby('objid').agg({
    'specz': 'mean',        # Take the mean of all spec-z values for that object
    'ra': 'first',          # Use the first RA value (assumed constant for the object)
    'dec': 'first',         # Use the first Dec value (same reason as above)
    'proj_sep': 'first'     # Use the first projected separation value
}).reset_index()


In [ ]:
# Calculating the average specz for each id:
averaged_df = df.groupby('objid').agg({
    'specz': 'mean',
    'ra': 'first',
    'dec': 'first',
    'proj_sep': 'first'
}).reset_index()

print(f"After averaging: {len(averaged_df)} unique objects")
print("\nRedshift statistics:")
print(averaged_df.describe()['specz'])

To create a cut in the redshift so that a cluster can be identified. We must use some logic. Most astronomers prefer anything beyond 3*sigma away from the mean to be not part of the same group. 

Find the mean, standard deviation and limits of the redshift from the data

In [ ]:
# Calculate mean, standard deviation and 3-sigma limits
mean_redshift = averaged_df['specz'].mean()
std_redshift = averaged_df['specz'].std()
lower_limit = mean_redshift - 3 * std_redshift
upper_limit = mean_redshift + 3 * std_redshift

print(f"Mean redshift: {mean_redshift:.4f}")
print(f"Standard deviation: {std_redshift:.4f}")
print(f"3-sigma lower limit: {lower_limit:.4f}")
print(f"3-sigma upper limit: {upper_limit:.4f}")

You can also use boxplot to visualize the overall values of redshift 

In [ ]:
# Plot the distribution of redshift as histogram and a boxplot 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Boxplot
ax1.boxplot(averaged_df['specz'])
ax1.set_ylabel('Redshift')
ax1.set_title('Boxplot of Redshift Distribution')
ax1.grid(True, alpha=0.3)

# Histogram
ax2.hist(averaged_df['specz'], bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(mean_redshift, color='red', linestyle='--', label=f'Mean: {mean_redshift:.3f}')
ax2.axvline(lower_limit, color='orange', linestyle='--', label=f'3σ limits')
ax2.axvline(upper_limit, color='orange', linestyle='--')
ax2.set_xlabel('Redshift')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Redshift for This Data')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

But the best plot would be a histogram to see where most of the objects downloaded lie in terms of redshift value

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(averaged_df['specz'], bins=90, alpha=0.7, edgecolor='black')
plt.axvline(mean_redshift, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_redshift:.3f}')
plt.axvline(lower_limit, color='orange', linestyle='--', linewidth=2, label='3σ limits')
plt.axvline(upper_limit, color='orange', linestyle='--', linewidth=2)
plt.xlabel('Redshift')
plt.ylabel('Number of Objects')
plt.title('Detailed Histogram of Redshift Distribution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

Filter your data based on the 3-sigma limit of redshift. You should remove all data points which are 3-sigma away from mean of redshift

In [ ]:
# Filtering the data based on specz values, used 3 sigma deviation from mean as upper limit.
filtered_df = averaged_df[
    (averaged_df['specz'] >= lower_limit) & 
    (averaged_df['specz'] <= upper_limit)
].copy()

print(f"Original data: {len(averaged_df)} objects")
print(f"After 3σ filtering: {len(filtered_df)} objects")
print(f"Removed: {len(averaged_df) - len(filtered_df)} objects ({100*(len(averaged_df) - len(filtered_df))/len(averaged_df):.1f}%)")

Use the relation between redshift and velocity to add a column named velocity in the data. This would tell the expansion velocity at that redshift 

In [ ]:
# Calculate velocity using Hubble's law: v = H_0 * c * z / (1 + z) for relativistic correction
# For small redshifts, v ≈ c * z
filtered_df['velocity'] = c_light * filtered_df['specz']  # km/s

print("Velocity statistics (km/s):")
print(filtered_df['velocity'].describe())

In [ ]:
# Plot the velocity column created as hist
plt.figure(figsize=(10, 6))
plt.hist(filtered_df['velocity'], bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Velocity (km/s)')
plt.ylabel('Number of Objects')
plt.title('Distribution of Recession Velocities')
plt.grid(True, alpha=0.3)
plt.show()

use the dispersion equation to find something called velocity dispersion. You can even refer to wikipedia to know about the term [wiki link here](https://en.wikipedia.org/wiki/Velocity_dispersion#:~:text=In%20astronomy%2C%20the%20velocity%20dispersion,%2C%20galaxy%20cluster%2C%20or%20supercluster.)

It is the velocity dispersion value which tells us, some galaxies might be part of even larger groups!!

### Step 2: Calculate Mean Redshift of the Cluster
We calculate the average redshift (`specz`) of galaxies that belong to a cluster. This gives us an estimate of the cluster's systemic redshift.

`cluster_redshift = filtered_df['specz'].mean()`


The velocity dispersion \( v \) of galaxies relative to the cluster mean redshift is computed using the relativistic Doppler formula:

$$
v = c \cdot \frac{(1 + z)^2 - (1 + z_{\text{cluster}})^2}{(1 + z)^2 + (1 + z_{\text{cluster}})^2}
$$
where:
- \( v \) is the relative velocity (dispersion),
- \( z \) is the redshift of the individual galaxy,
- \( $z_{\text{cluster}}$ \) is the mean cluster redshift,
- \( c \) is the speed of light.


In [ ]:
# Calculate mean cluster redshift
cluster_redshift = filtered_df['specz'].mean()

# Calculate velocity dispersion using relativistic Doppler formula
z = filtered_df['specz'].values
z_cluster = cluster_redshift

# Relativistic velocity dispersion
v_dispersion = c_light * ((1 + z)**2 - (1 + z_cluster)**2) / ((1 + z)**2 + (1 + z_cluster)**2)
filtered_df['v_dispersion'] = v_dispersion

# Calculate the characteristic velocity dispersion (standard deviation)
disp = np.std(v_dispersion)

print(f"Cluster mean redshift: {cluster_redshift:.4f}")
print(f"Velocity dispersion statistics (km/s):")
print(f"  Standard deviation (σ): {disp:.1f} km/s")
print(f"  Mean: {np.mean(v_dispersion):.1f} km/s")
print(f"  Median: {np.median(v_dispersion):.1f} km/s")

Pro tip: Check what the describe function of pandas does. Does it help to get quick look stats for your column of dispersion??

In [ ]:
# Using pandas describe function for quick statistics
print("Velocity dispersion statistics using pandas describe():")
print(filtered_df['v_dispersion'].describe())

# Plot histogram of velocity dispersion
plt.figure(figsize=(10, 6))
plt.hist(filtered_df['v_dispersion'], bins=30, alpha=0.7, edgecolor='black')
plt.axvline(0, color='red', linestyle='--', label='Zero velocity')
plt.axvline(disp, color='orange', linestyle='--', label=f'σ = {disp:.1f} km/s')
plt.axvline(-disp, color='orange', linestyle='--')
plt.xlabel('Velocity Dispersion (km/s)')
plt.ylabel('Number of Objects')
plt.title('Velocity Dispersion Distribution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
print(f"The value of the cluster redshift = {cluster_redshift:.4f}")
print(f"The characteristic value of velocity dispersion of the cluster along the line of sight = {disp:.1f} km/s.")

### Step 4: Visualizing Angular Separation of Galaxies
We plot a histogram of the projected (angular) separation of galaxies from the cluster center. This helps us understand the spatial distribution of galaxies within the cluster field.

- The x-axis represents the angular separation (in arcminutes or degrees, depending on units).
- The y-axis shows the number of galaxies at each separation bin.



In [ ]:
# Plot histogram for proj_sep column
plt.figure(figsize=(10, 6))
plt.hist(filtered_df['proj_sep'], bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Projected Separation (arcmin)')
plt.ylabel('Number of Objects')
plt.title('Distribution of Projected Separations from Cluster Center')
plt.grid(True, alpha=0.3)
plt.show()

print("Projected separation statistics:")
print(filtered_df['proj_sep'].describe())

### Determining size and mass of the cluster:

### Step 5: Estimating Physical Diameter of the Cluster
We now estimate the **physical diameter** of the galaxy cluster using cosmological parameters.

- `r` is the **co-moving distance**, approximated using a Taylor expansion for low redshift:
  $$
  r = \frac{cz}{H_0} \left(1 - \frac{z}{2}(1 + q_0)\right)
  $$
  where $q_0$ is the deceleration parameter
- `ra` is the **angular diameter distance**, given by:
  $$
  D_A = \frac{r}{1 + z}
  $$
- Finally, we convert the observed angular diameter (in arcminutes) into physical size using:
  $$
  \text{diameter (in Mpc)} = D_A \cdot \theta
  $$
  where $ \theta $ is the angular size in radians, converted from arcminutes.

> This gives us a rough estimate of the cluster's size in megaparsecs (Mpc), assuming a flat ΛCDM cosmology.


In [ ]:
# Calculate comoving distance using Taylor expansion
z_cluster = cluster_redshift
r_comoving = (c_light * z_cluster / H_0) * (1 - (z_cluster/2) * (1 + q0))  # Mpc

# Calculate angular diameter distance
D_A = r_comoving / (1 + z_cluster)  # Mpc

# Convert maximum projected separation to physical diameter
# Assuming the cluster extends to the maximum observed separation
max_sep_arcmin = filtered_df['proj_sep'].max()
max_sep_radians = max_sep_arcmin * (np.pi / 180) / 60  # Convert arcmin to radians

# Physical diameter in Mpc
diameter = 2 * D_A * max_sep_radians  # Factor of 2 for diameter

print(f"Cluster redshift: {z_cluster:.4f}")
print(f"Comoving distance: {r_comoving:.2f} Mpc")
print(f"Angular diameter distance: {D_A:.2f} Mpc")
print(f"Maximum projected separation: {max_sep_arcmin:.2f} arcmin")
print(f"Estimated cluster diameter: {diameter:.2f} Mpc")

### Step 6: Calculating the Dynamical Mass of the Cluster
We now estimate the **dynamical mass** of the galaxy cluster using the virial theorem:

$$
M_{\text{dyn}} = \frac{3 \sigma^2 R}{G}
$$

Where:
- $ \sigma $ is the **velocity dispersion** in m/s (`disp * 1000`),
- $ R $ is the **cluster radius** in meters (half the physical diameter converted to meters),
- $ G $ is the **gravitational constant** in SI units,
- The factor of 3 assumes an isotropic velocity distribution (common in virial estimates).

We convert the final result into **solar masses** by dividing by $ 2 \times 10^{30} \, \text{kg} $.

> This mass estimate assumes the cluster is in dynamical equilibrium and bound by gravity.


In [ ]:
# Calculating the dynamical mass in solar masses:
# Convert units properly
sigma_ms = disp * 1000  # Convert km/s to m/s
R_meters = (diameter * 0.5) * 3.086e22  # Convert Mpc to meters (1 Mpc = 3.086e22 m)
G_SI = 6.674e-11  # Gravitational constant in SI units (m^3 kg^-1 s^-2)
M_sun = 1.989e30  # Solar mass in kg

# Calculate dynamical mass
M_dyn = (3 * sigma_ms**2 * R_meters) / G_SI  # Mass in kg
M_dyn_solar = M_dyn / M_sun  # Convert to solar masses

print(f"Velocity dispersion: {disp:.1f} km/s = {sigma_ms:.0f} m/s")
print(f"Cluster radius: {diameter/2:.2f} Mpc = {R_meters:.2e} m")
print(f"Dynamical Mass of the cluster is {M_dyn_solar:.2e} solar masses")
print(f"That's approximately {M_dyn_solar/1e14:.1f} × 10^14 solar masses")

### Summary of Results

In [ ]:
print("=" * 50)
print("GALAXY CLUSTER ANALYSIS SUMMARY")
print("=" * 50)
print(f"Number of galaxies analyzed: {len(filtered_df)}")
print(f"Cluster redshift: {cluster_redshift:.4f}")
print(f"Velocity dispersion: {disp:.1f} km/s")
print(f"Cluster diameter: {diameter:.2f} Mpc")
print(f"Dynamical mass: {M_dyn_solar:.2e} M☉")
print(f"Mass-to-light ratio implications: This is a typical galaxy cluster mass")
print("=" * 50)